# Prepare Statistical Yearbook data

## Load necessary python libraries and define working directory

This script will heavily rely on the data transformation and data management methods available from the `pandas` python library.

In [3]:
import json
import urllib3  # allows to access a URL with python
import pandas as pd
import math
import os 
import hashlib

dir_path = os.path.dirname(os.path.realpath('__file__'))
print(dir_path)

data_dir = r'../data/'
print('data inputs dir: ' + data_dir)

output_dir = r'../output/'
print('outputs dir: ' + output_dir)


# https://volderette.de/jupyter-notebook-tip-multiple-outputs/
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

C:\Users\L.GonzalezMorales\Documents\GitHub\NepalCBS\scripts
data inputs dir: ../data/
outputs dir: ../output/


## Utilities

#### Compute a hash of a dictionary

In [4]:
def dict_hash(d):
    out = hashlib.md5()
    for key, value in d.items():
        out.update(key.encode('utf-8'))
        out.update(str(value).encode('utf-8'))
    return out.hexdigest()


#### Get unique dictionaries in a list

In [5]:
def unique_dicts(dictionary_list):

    uniques_map = {}

    for d in dictionary_list:
        uniques_map[dict_hash(d)] = d

    return list(uniques_map.values())


#### Extract subset of key-value pairs from Python dictionary object

In [6]:
def subdict_list(dict_list, keys_list, exclude = False):
    sub_d_list = []
    if exclude:
        for d in dict_list:
            sub_d= {k: d[k] for k in d.keys() if k not in keys_list}
            sub_d_list.append(sub_d)
    else:
        for d in dict_list:
            sub_d= {k: d[k] for k in keys_list}
            sub_d_list.append(sub_d)
    
    return sub_d_list




#### Get a dict from a list based on something inside the dict

In [7]:
def select_dict(dict_list, k, v):
    selected = []
    for d in dict_list:
        if d[k] == v:
            selected.append(d)
    return selected

## Read code lists

In [32]:
xls = pd.ExcelFile(data_dir + 'Yearbook - Code Lists.xlsx')

code_lists = dict(pd.read_excel(xls, None))

code_lists_catalogue = code_lists.keys()
display(code_lists_catalogue)

code_lists['CL_INDICATOR']

dict_keys(['CL_INDICATOR', 'CL_REF_AREA', 'CL_SEX', 'CL_URBANIZATION', 'CL_AGE', 'CL_UNIT_MEASURE', 'CL_RELIGION', 'CL_MOTHER_TONGUE', 'CL_CASTE_ETHNICGROUP', 'CL_LITERACY', 'CL_DISABILITY_STATUS', 'CL_ECON_ACTIVITY_STATUS', 'CL_ACTIVITY', 'CL_OCCUPATION'])

Code             Description
0      YB_NO_HH    Number of households
1  YB_AVG_HH_SZ  Average household size
2        YB_POP              Population
3   YB_POP_DENS      Population density
4       YB_AREA                    Area
5   YB_NO_WARDS         Number of wards

## Read data

In [55]:
xls = pd.ExcelFile(data_dir + 'Yearbook - Ch01.xlsx')

chapter01 = dict(pd.read_excel(xls, None))

chapter01_tables = list(chapter01.keys())
display(chapter01_tables)

['1.1',
 '1.2',
 '1.3',
 '1.4',
 '1.5',
 '1.6',
 '1.7',
 '1.8',
 '1.9',
 '1.10',
 '1.11',
 '1.12',
 '1.13',
 '1.14',
 '1.15',
 '1.16',
 'List of concepts']

In [50]:
chapter01['1.1'].head(6)

INDICATOR        INDICATOR_DESC REF_AREA REF_AREA_DESC  TIME_PERIOD SEX  \
0  YB_NO_HH  Number of households       NP         Nepal         2011  _T   
1  YB_NO_HH  Number of households       NP         Nepal         2011  _T   
2  YB_NO_HH  Number of households       NP         Nepal         2011  _T   
3  YB_NO_HH  Number of households   NP_MTN      Mountain         2011  _T   
4  YB_NO_HH  Number of households   NP_HIL          Hill         2011  _T   
5  YB_NO_HH  Number of households   NP_TER         Terai         2011  _T   

                  SEX_DESC URBANIZATION URBANIZATION_DESC AGE  ...  \
0  Total or not applicable           _T             Total  _T  ...   
1  Total or not applicable            U             Urban  _T  ...   
2  Total or not applicable            R             Rural  _T  ...   
3  Total or not applicable           _T             Total  _T  ...   
4  Total or not applicable           _T             Total  _T  ...   
5  Total or not applicable           _T             Total  _T  ...   

                       OCCUPATION_DESC ACTIVITY ACTIVITY_DESC  OBS_VALUE  \
0  Total or no breakdown by Occupation       _T  No breakdown  5427302.0   
1  Total or no breakdown by Occupation       _T  No breakdown  1047297.0   
2  Total or no breakdown by Occupation       _T  No breakdown  4380005.0   
3  Total or no breakdown by Occupation       _T  No breakdown   364120.0   
4  Total or no breakdown by Occupation       _T  No breakdown  2534430.0   
5  Total or no breakdown by Occupation       _T  No breakdown  2528752.0   

     Ref_Area_Type UNIT_MEASURE UNIT_MEASURE_DESC UNIT_MULT  \
0              NaN       NUMBER            Number         0   
1              NaN       NUMBER            Number         0   
2              NaN       NUMBER            Number         0   
3  Ecological Belt       NUMBER            Number         0   
4  Ecological Belt       NUMBER            Number         0   
5  Ecological Belt       NUMBER            Number         0   

                              FOOTNOTE                       SOURCE  
0  Includes institutional households    Central Bureau of Statitics  
1  Includes institutional households    Central Bureau of Statitics  
2  Includes institutional households    Central Bureau of Statitics  
3  Includes institutional households    Central Bureau of Statitics  
4  Includes institutional households    Central Bureau of Statitics  
5  Includes institutional households    Central Bureau of Statitics  

[6 rows x 34 columns]

# Read coordinates

In [136]:
with open(data_dir + 'ref_areas/nepal_districts.geojson') as json_file:
    data = json.load(json_file)
    
districts = []
for f in data['features']:
    f['properties']['X'] = f['geometry']['coordinates'][0]
    f['properties']['Y'] = f['geometry']['coordinates'][1]
    districts.append(f['properties'])

display(districts[0])

df_districts = pd.DataFrame(districts)
df_districts = df_districts.rename(columns = {"Code": "REF_AREA",
                                              "Description":"REF_AREA_DESC"}) 
df_districts = df_districts.drop(['ObjectId'], axis=1)
df_districts.to_excel(data_dir + 'ref_areas/nepal_districts_xy.xlsx')
#display(df_districts)

with open(data_dir + 'ref_areas/nepal_development_regions.geojson') as json_file:
    data = json.load(json_file)
    
dev_regions = []
for f in data['features']:
    f['properties']['X'] = f['geometry']['coordinates'][0]
    f['properties']['Y'] = f['geometry']['coordinates'][1]
    dev_regions.append(f['properties'])

display(dev_regions[0])

df_devRegions = pd.DataFrame(dev_regions)
df_devRegions = df_devRegions.rename(columns = {"Code": "REF_AREA",
                                              "Description":"REF_AREA_DESC"}) 
df_devRegions = df_devRegions.drop(['ObjectId'], axis=1)
df_devRegions.to_excel(data_dir + 'ref_areas/nepal_devRegions_xy.xlsx')
#display(df_devRegions)


{'Parent': 'NP_DEV_EAST',
 'Code': '101',
 'Description': 'Taplejung',
 'Ref_Area_Type': 'District',
 'ISO': None,
 'X': 87.821800915844,
 'Y': 27.582127224225,
 'ObjectId': 1}

{'Parent': 'NP',
 'Code': 'NP_DEV_EAST',
 'Description': 'Eastern Dev. Region',
 'Description_2': 'Purwanchal',
 'Ref_Area_Type': 'Development Region',
 'ISO': 'NP-4',
 'ObjectId': 1,
 'X': 87.14802,
 'Y': 27.2399800000001}

In [69]:
district_codes = []
for d in districts:
    district_codes.append(d['Code'])
district_codes[0]

'101'

## Select district-level data and add coordinates

In [125]:


x = chapter01[chapter01_tables[0]]


  
x = x.astype(convert_dict) 
#print(x.dtypes) 

#list(x['REF_AREA'])

In [139]:

convert_dict = {'REF_AREA': str } 

for i in chapter01_tables:
    if i != 'List of concepts':

        x_district = chapter01[i].drop(['Ref_Area_Type'], axis=1)
        x_district = x_district.astype(convert_dict) 
        
        df_districts = df_districts.astype(convert_dict) 
        result = pd.merge(x_district, df_districts, on=['REF_AREA', 'REF_AREA_DESC'])
        
        if not result.empty:
            #result.to_excel(output_dir + 'Ch' + i.replace('.','_') + '_District' + '.xlsx')
            #display(result.head(5))

            
            writer = pd.ExcelWriter(output_dir + 'Ch' + i.replace('.','_') + '_District' + '.xlsx', engine='xlsxwriter')
            result.to_excel(writer, index=False)
            writer.save() 
            
